In [3]:
import torch


In [18]:
torch.triu(torch.ones([2,1,3,3], dtype=torch.bool), diagonal=1)


tensor([[[[False,  True,  True],
          [False, False,  True],
          [False, False, False]]],


        [[[False,  True,  True],
          [False, False,  True],
          [False, False, False]]]])

In [19]:
a=torch.rand([2,3])
a[-2:,:]

tensor([[0.3552, 0.3986, 0.7716],
        [0.1178, 0.0323, 0.4546]])

In [20]:
a[:,-1:]

tensor([[0.7716],
        [0.4546]])

In [21]:
b=torch.rand([2,3,4])
c=torch.rand([2,6,4])
torch.cat((b,c),-2).shape

torch.Size([2, 9, 4])

In [26]:
a=torch.arange(3).unsqueeze(1)
a

tensor([[0],
        [1],
        [2]])

In [28]:
torch.randint(2, (6, 4))

tensor([[1, 0, 1, 1],
        [1, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 0, 0, 1],
        [1, 1, 1, 1],
        [0, 1, 1, 1]])

In [32]:
V=torch.rand([2,3,4,5])
L_Q=6
B, H, L_V, D = V.shape
V_sum = V.mean(dim=-2)
contex = V_sum.unsqueeze(-2).expand(B, H, L_Q, V_sum.shape[-1]).clone()

In [18]:
K=torch.rand([2,3,40,50])
Q=torch.rand([2,3,40,50])
sample_k=10
n_top=15

B, H, L_K, E = K.shape
_, _, L_Q, _ = Q.shape

# calculate the sampled Q_K
K_expand = K.unsqueeze(-3).expand(B, H, L_Q, L_K, E)
index_sample = torch.randint(L_K, (L_Q, sample_k))  # real U = U_part(factor*ln(L_k))*L_q
K_sample = K_expand[:, :, torch.arange(L_Q).unsqueeze(1), index_sample, :]
Q_K_sample = torch.matmul(Q.unsqueeze(-2), K_sample.transpose(-2, -1)).squeeze()

# find the Top_k query with sparisty measurement
M = Q_K_sample.max(-1)[0] - torch.div(Q_K_sample.sum(-1), L_K)
M_top = M.topk(n_top, sorted=False)[1]

# use the reduced Q to calculate Q_K
Q_reduce = Q[torch.arange(B)[:, None, None],
            torch.arange(H)[None, :, None],
            M_top, :]  # factor*ln(L_q)
Q_K = torch.matmul(Q_reduce, K.transpose(-2, -1))  # factor*ln(L_q)*L_k


In [16]:
print(K_expand.shape)
print(index_sample.shape)
print(K_sample.shape)
print(Q_K_sample.shape)
print(Q_K.shape)
print(M_top.shape)

torch.Size([2, 3, 40, 40, 50])
torch.Size([40, 10])
torch.Size([2, 3, 40, 10, 50])
torch.Size([2, 3, 40, 10])
torch.Size([2, 3, 15, 40])
torch.Size([2, 3, 15])


In [12]:
V=torch.rand([2,3,40,50])
B, H, L_V, D = V.shape
# V_sum = V.sum(dim=-2)
V_sum = V.mean(dim=-2)
contex = V_sum.unsqueeze(-2).expand(B, H, L_Q, V_sum.shape[-1]).clone()

In [14]:
print(V_sum.shape)
print(contex.shape)

torch.Size([2, 3, 50])
torch.Size([2, 3, 40, 50])


In [21]:
B, H, L_V, D = V.shape
context_in=contex
index=M_top
attn = torch.softmax(Q_K, dim=-1)  # nn.Softmax(dim=-1)(scores)

context_in[torch.arange(B)[:, None, None],
torch.arange(H)[None, :, None],
index, :] = torch.matmul(attn, V).type_as(context_in)

In [24]:
print(context_in.shape)
print(attn.shape)
print(V.shape)

torch.Size([2, 3, 40, 50])
torch.Size([2, 3, 15, 40])
torch.Size([2, 3, 40, 50])


In [26]:
context_in[torch.arange(B)[:, None, None],
torch.arange(H)[None, :, None],
index, :].shape

torch.Size([2, 3, 15, 50])

In [28]:
torch.matmul(attn, V).type_as(context_in).shape

torch.Size([2, 3, 15, 50])

In [4]:
import math
max_len=5000
d_model=8

pe = torch.zeros(max_len, d_model).float()
pe.require_grad = False

position = torch.arange(0, max_len).float().unsqueeze(1)
div_term = (torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)).exp()

pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)

pe = pe.unsqueeze(0)

In [6]:
pe.shape

torch.Size([1, 5000, 8])

In [1]:
import numpy as np 
pred=np.load('./results/check_informer_ETTh1_ftM_sl96_ll48_pl336_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_Exp_0/pred.npy')

FileNotFoundError: [Errno 2] No such file or directory: './results/check_informer_ETTh1_ftM_sl96_ll48_pl336_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_Exp_0/pred.npy'

In [6]:
import pandas as pd
root_path='../data/ETT/'
data_path='ETTh1.csv'
df_raw = pd.read_csv(os.path.join(root_path,data_path))
cols_data = df_raw.columns[1:]
df_data = df_raw[cols_data]
data = df_data.values

In [10]:
seq_len=96
border1s = [0, 12 * 30 * 24 - seq_len, 12 * 30 * 24 + 4 * 30 * 24 - seq_len]
border2s = [12 * 30 * 24, 12 * 30 * 24 + 4 * 30 * 24, 12 * 30 * 24 + 8 * 30 * 24]
set_type=0
border1 = border1s[set_type]
border2 = border2s[set_type]
data_x = data[border1:border2]
data_y = data[border1:border2]

In [13]:
print(data_x.shape)
print(data.shape)

(8640, 7)
(17420, 7)
